## Purpose: Clean and Feature Engineer for a quote to order model and a order to shipment quantity model that will generalize to inputs beyond range of dataset.  
#### A closed order is a quote that results in an order within 90 days from the quote date.  
#### A shipment is a sale during the 1 year period after the quote closes to an order.

### This model is designed to work with sparse data and high granularity.  In this case, granulaty is from many part numbers in combination with many customers with intermittent purchases.  No complete time sequences exist due to the granularity.  It can be applied to retail and distribution models as well as demand planning systems.

### Libraries and Utilities

In [1]:
%reload_ext autoreload
%autoreload

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#allow cell to perform multiple computations

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
#set cell width as percentage of window

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
from tensorflow.keras.backend import clear_session
clear_session()

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn_pandas import DataFrameMapper 
import os as os
import string
from joblib import dump, load
import time
import gc

###  Data cleaning functions ('parsers')

In [7]:
# Function to strip string of leading or following blanks and remove all blank entries ('')

def strp_by_col(df,col):#col is with string notation
    df[col]=df[col].apply(lambda x: x.lstrip() if type(x)==str else x)
    df[col]=df[col].apply(lambda x: x.rstrip() if type(x)==str else x)
    df = df[df[col].notnull()] 
    return df

In [8]:
# Function to clean integers with other characters inserted - i.e., keyboard error 

def clean_ints(entry):
    ltr_chars = string.ascii_letters #all letter characters
    punct_chars = string.punctuation #all punctuation
    proxies = ['!' for ltr in ltr_chars]
    proxy_chars = ''.join(map(str,proxies))
    puncts = ['!' for p in punct_chars]
    proxy_puncts = ''.join(map(str,puncts))
    ltr_cleaner = str.maketrans(ltr_chars,proxy_chars)
    entry_ltr = entry.translate(ltr_cleaner)
    punct_cleaner = str.maketrans(punct_chars,proxy_puncts)
    entry_clean = entry_ltr.translate(punct_cleaner)
    entry_clean = entry_clean.replace('!','')
    return entry_clean

In [9]:
# Function to strip integer objects of leading spaces, trailing spaces, leading 0, empty strings and clean keyboard errors

def str_to_int(db,col):#col is WITH string notation
    db[col]=db[col].apply(lambda x: str(x))
    db[col]=db[col].apply(lambda x: x.strip())
    db[col]=db[col].apply(lambda x: x.lstrip())
    db[col]=db[col].apply(lambda x: x.rstrip())
    db[col]=db[col].apply(lambda x: clean_ints(x))
    db[col]=db[col].apply(lambda x: int(x) if x.isdigit() else None) # to remove empty strings
    db = db[db[col].notnull()] #remove rows where cannot convert object to integer
    return db

In [10]:
# Function to clean floats of inserted characters and punctuation by keyboard error 

def clean_floats(entry):
    ltr_chars = string.ascii_letters #all letter characters
    punct_chars = string.punctuation #all punctuation
    punct_chars = punct_chars[:13]+punct_chars[15:] # punctuation with decimal point excluded
    proxies = ['!' for ltr in ltr_chars]
    proxy_chars = ''.join(map(str,proxies))
    puncts = ['!' for p in punct_chars]
    proxy_puncts = ''.join(map(str,puncts))
    ltr_cleaner = str.maketrans(ltr_chars,proxy_chars)
    entry_ltr = entry.translate(ltr_cleaner)
    punct_cleaner = str.maketrans(punct_chars,proxy_puncts)
    entry_clean = entry_ltr.translate(punct_cleaner)
    entry_clean = entry_clean.replace('!','')
    return entry_clean

In [11]:
# Function to strip float objects of leading or trailing spaces, leading 0, remove '' and clean keyboard errors

def str_to_flt(db,col):#col is WITH string notation
    db[col]=db[col].apply(lambda x: str(x))
    db[col]=db[col].apply(lambda x: x.strip())
    db[col]=db[col].apply(lambda x: x.lstrip())
    db[col]=db[col].apply(lambda x: x.rstrip())
    db[col]=db[col].apply(lambda x: clean_floats(x))
    db[col]=db[col].apply(lambda x: float(x) if x != '' else None)
    db = db[db[col].notnull()] # remove rows where cannot convert object to float
    return db

In [12]:
# Function to reduce memory used by integers and floats

def reduce_memory_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df.loc[:,col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df.loc[:,col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df.loc[:,col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df.loc[:,col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df.loc[:,col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df.loc[:,col] = df[col].astype(np.float32)
                else:
                    df.loc[:,col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [13]:
# DOMAIN KNOWLEDGE:  ship_date is date of a shipment ordered from a valid quote
# DOMAIN KNOWLEDGE:  there can be multiple shipments (i.e., multiple ship_date's) against a quote while order is valid
# DOMAIN KNOWLEDGE:  quotes are open for ninety days and require a shipment to remain valid
# DOMAIN KNOWLEDGE:  quotes remain valid for one year after first ship_date

In [14]:
# DOMAIN KNOWLEDGE: Win is a quote with a shipment against the quote while quote is in 90 day open period

### Load And Clean Data

### Definitions:
####   'quote' is a unique number assigned to every quote; more than one part can be included on a quote number
####   'part' is the part name being quoted
####   'plant' is the company division that manufactures the part
####   'dist' is a unique number assigned to the company distributors
####   'region' is the geographical region 
####   'quote_qty' is the requested quote amount
####   'sales_qty' is the quantity shipped on the 'ship_date'; there can be multiple shipments against a quote
####   'quote_price' is the price first quoted; shipping prices can vary from competitive requirements
####   'ship_date' are the dates of shipment
####   'quote_date' is the date of the quote

In [15]:
# Load order data
# path to data file
path = os.getcwd()
file = '/order_demo_data.csv'
filepath = path + file

# columns to be read into training and prediction data
cols = ['quote', 'part', 'plant', 'dist', 'region', 'quote_qty', 'sales_qty',
       'quote_pr', 'ship_date', 'quote_date']

# read in data and parse relevant dates
dp=pd.read_csv(filepath, usecols=cols, parse_dates=['ship_date','quote_date'])


In [16]:
# Limit historical data to last four years on quote date:  predictions with new data is then one month rolling
            # and fit on new data can use previously fit weights from the model
    
# DOMAIN KNOWLEDGE: prior to this date the client deems the data unreliable

# For this exercise set today's date as latest date of quote_date; in production the date would be the current date
            # today_date = dt.today() in production version
today_date = dp.quote_date.max()

# Set earliest date as 48 months previous to current date - any quote before this date is deleted
first_date = today_date - pd.DateOffset(months=48)

# Keep quote data from today_date to prior 48 months first date
dp = dp[(dp['quote_date'] >= first_date)]

#### Assuring data is clean

In [17]:
# Remove any quote dates == NaT
dp = dp[dp.quote_date.notnull()]

# Convert integer columns to dtype int  - drop ''
dp = str_to_int(dp,'quote')
dp = str_to_int(dp,'dist')
dp = str_to_int(dp,'quote_qty')
dp = str_to_int(dp,'sales_qty')

# Convert float or money columns to dtype float - drop ''
dp = str_to_flt(dp,'quote_pr')

# Strip leading and trailing blanks from string columns
dp = strp_by_col(dp,'plant')
dp = strp_by_col(dp,'region')

# Reduce memory usage from integers and floats
dp = reduce_memory_usage(dp)

### Apply domain knowledge to data

In [18]:
# DOMAIN KNOWLEDGE: negative or zero prices and quantities are either samples or for accounting purposes only

# drop zero or negative prices and quantity
dp = dp.query('quote_pr > 0 & quote_qty > 0 & sales_qty >= 0')

In [20]:
# DOMAIN KNOWLEDGE: prices above $1 per unit are special order designs or accounting adjustments

# drop prices over a fixed dollar amount of p
p = 1
dp = dp[dp['quote_pr'] <= p]

In [22]:
# Function to clip by percentile of number of events

def clip_by_value_count(df,col,limit): #col requires string notation
 
    # Build value_counts dataframe
    temp = df[col].value_counts()
    temp = pd.DataFrame(temp).reset_index()
    temp.columns = [col,'count']

    # Computing cumulative percentage
    temp['cum_percent'] = 100*(temp['count'].cumsum() / temp['count'].sum())

    # Calculate data at clip  limit
    temp = temp[temp['cum_percent'] <= limit]
    item_list = list(temp[col])
    del temp

    # Return clipped data
    return df[df[col].isin(item_list)]


In [23]:
# DOMAIN KNOWLEDGE: parts with few transactions are special order designs

# this clips the parts quoted to those who make up 80% of the quote transactions
dp = clip_by_value_count(dp, 'part', 80)

# DOMAIN KNOWLEDGE: distributors with few transactions are soliciting sample quantities or second sourcing small quantities

# this clips the distributors to those who make up 80% of the quote transactions
dp = clip_by_value_count(dp, 'dist', 80)

# DOMAIN KNOWLEDGE: quote qty of less than 200 are courtesy quotes for sales demontration purposes

dp = dp[dp.quote_qty > 200]

## Identify Closed, Pending and Expired Quotes by Line Item
#### Aggregate by quote number and part number to remove repeated ship dates for a single order/line combination
#### Each combination of quote and line are a separate 'quote' that remain 'open' for ninety days
#### A 'closed quote' is a quote and line item combination that received a shipping order.  A 'pending quote' is a quote and line item combination that is still within the 90 day period where quotes remain 'open'.  An 'expired quote' is a quote and line item combination that did not received a shipping order before the open period expired. 
#### The purpose here is to remove repeated ship dates inflating order closing and to identify closed, pending and expired quotes.

In [24]:
# Aggregate to remove multiple shipment dates for a quote and line combination - each line is a unique material
# Sum the sold quantity to obtain total quantity sold on the quote for each quote and part
# Note that sales_qty summed will convert nan's to 0
grps = ['quote','part']  #multiple parts are multiple line items
aggs = {'plant':['last'], 'dist':['last'],
       'region':['last'], 'quote_qty':['last'], 'sales_qty':['sum'],
       'quote_pr':['mean'], 'ship_date':['first'], 'quote_date':['first']}#one dist_num,so_name,div per quote
dx = dp.groupby(grps).agg(aggs).reset_index() 

# flatten multiple index on columns
dx.columns = ['_'.join(col) for col in dp.columns.values]

# rename columns
dx.columns = list(dp.columns)

In [25]:
# DOMAIN KNOWLEDGE:  Top 20% of quote qty receive special management attn for contracting and are not part of the
                     #daily operating pricing - or extremely high quote qty are intentional 
                     #operator error trying to override system

# Cap and floor the quote quantity related to the sales quantity
# Use the 'capture rate' to measure the relationship between both qty's: capture rate = sales quantity/quote quantity

# Insert capture rate into dataframe
dx['Capture'] = dx.sales_qty/dx.quote_qty

# The 'capture at percentile rank' = limit is the limit 'l'
# The median 'capture at percentile rank' = 50 is the median 'm'
# Incrs quote quantity to capture at percentile rank * sales quantity when quote_qty less than at 1/p times sales_qty

def modify_quote_quantity(dx,target=80):
    # Find target percentile of capture rates
    dc = dx[dx.Capture > 0]
    ul = round(np.percentile(np.array(dc['Capture']).reshape(-1,1),target),1)
    ll = round(np.percentile(np.array(dc['Capture']).reshape(-1,1),100-target),1)
    
    # increase quote_qty to sales_qty times 1/ul when sales_qty is greater than ul times quote_qty
    dx['quote_qty'] = np.where(dx['sales_qty'] >= ul * dx['quote_qty'],dx['sales_qty'] * 1/ul,dx['quote_qty'])
    
    # decrease quote_qty to sales_qty times ll when sales_qty is less than ll time sales quantity
    dx['quote_qty'] = np.where((dx['sales_qty'] <= ll * dx['quote_qty']) & (dx['sales_qty'] > 0),dx['sales_qty'] * ll,dx['quote_qty'])
        
    # convert quote_qty back to ints
    dx['quote_qty'] = dx['quote_qty'].apply(lambda q: np.int64(q))
    return dx


dx = modify_quote_quantity(dx)

# Drop 'Capture' from data
dx.drop(['Capture'],axis=1,inplace=True)

In [26]:
# Create dataframes of closed, pending and open quotes

# Set current and pending dates

# For this exercise set today's date as latest date of quote_date; in production the date would be the current date
# today_date = dt.today() in production version
today_date = dx.quote_date.max()

# Set pending date as ninety days previous to current date - any quote on or after this date is still open
pending_date = today_date - pd.DateOffset(days=90)

# Filter closed and lost quotes
closed = dx[dx['sales_qty'] > 0] 
#pending = dx[(dx['sales_qty'] == 0) & (dx['quote_date'] >= pending_date)]
lost = dx[(dx['sales_qty'] == 0) & (dx['quote_date'] < pending_date)]

# Concatenate training data of closed and lost quotes
dx = pd.concat([closed,lost], axis=0, ignore_index=True)
dx.reset_index(drop=True,inplace=True)

In [27]:
# Create Wins for closed and lost orders not still pending
dx['Win'] = [1 if x > 0 else 0 for x in dx.sales_qty]

### Add date features

In [28]:
# Function to convert dates to categorical variables
# Monthly data is best fit to only Month, Quarter and Year variable

def add_date_features(data,date,name):
    data[name + 'Yr'] = data[date].dt.year
    #data[name + 'DAY'] = data[date].dt.dayofyear
    #data[name + 'WK] = data[date].dt.week
    data[name + 'Mon'] = data[date].dt.month 
    data[name + 'Qtr'] = data[date].dt.quarter
    #data.drop([date], axis = 1, inplace = True)
    return data

In [29]:
# Add categorical date features and save for future use

dx = add_date_features(dx,'quote_date','Quote_')
dx = reduce_memory_usage(dx)

### This data is developed for the quote closing predictor.

In [30]:
# Save features and targets for quote to order predictor
dump (dx,'Quote_Order_Data')

['Quote_Order_Data']

### The quantity predictor requires additional feature engineering
#### The ratio of quantity quoted to quantity shipped has extreme anomalies above the 95th percentile
#### The quantity predictor only uses the closed quotes to predict quantity shipped
#### The data contains shipping quantities for quotes not having reached the closing period of 12 months after the first shipdate.  

In [31]:
# Filter the data to only include closed quotes ('Wins')
ds = dx[dx.Win == 1]

In [32]:
# Compute months elapsed in shipment closing period (12 months after first ship date)

# Compute closing date
ds['Close_Date'] = ds.ship_date + pd.DateOffset(months=12)

# Compute months transpired in shipment closing period
ds['Months_Used'] = ((ds.Close_Date - today_date)/np.timedelta64(1,'M')).apply(lambda t: np.int(t))

# Convert 0 or negative months to 12
ds['Months_Used'] = ds['Months_Used'].apply(lambda t: 12 if t <= 0 else t)


In [33]:
# Sort values by year, qtr and month for convenience
ds = ds.sort_values(by=['Quote_Yr','Quote_Qtr','Quote_Mon'], ascending=True)

In [34]:
# Store order to ship data for estimator
ds = reduce_memory_usage(ds)
dump (ds,'Order_Ship_Data')

['Order_Ship_Data']

In [35]:
clear_session(); gc.collect()

293